In [1]:
import numpy as np
import pandas as pd

In [2]:
#For text preprocessing
import re
import spacy
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

In [3]:
# to suppress warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [4]:
# Importing Gensim
import gensim
from gensim import corpora

In [5]:
df=pd.read_csv('Assignment 2_Web Data.csv')

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SNO                 10000 non-null  int64 
 1   Date                10000 non-null  object
 2   Time                10000 non-null  object
 3   Media Type          10000 non-null  object
 4   Site Name           10000 non-null  object
 5   Site Domain         10000 non-null  object
 6   Mention URL         8308 non-null   object
 7   Publisher Name      9257 non-null   object
 8   Publisher Username  7752 non-null   object
 9   title               9980 non-null   object
 10  Mention Content     9996 non-null   object
 11  Country             10000 non-null  object
 12  State               5389 non-null   object
 13  City                3478 non-null   object
 14  Language            10000 non-null  object
dtypes: int64(1), object(14)
memory usage: 1.1+ MB


In [6]:
df.head()

,SNO,Date,Time,Media Type,Site Name,Site Domain,Mention URL,Publisher Name,Publisher Username,title,Mention Content,Country,State,City,Language
0,1,2021-03-15,23:41:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,90Hz #motog10power #motog30 #AsliAllRounderCha...,India,NaN,NaN,English
1,2,2021-03-15,17:27:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/RajibGogoi668/status/137143...,Rajib Gogoi,RajibGogoi668,RT @ManojSaru: Another Giveaway for #TechGyan ...,RT @ManojSaru: Another Giveaway for #TechGyan ...,India,Assam,Jorhat,English
2,3,2021-03-15,21:03:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,Wow Magnificent With Amazing Tech Features #Po...,India,NaN,NaN,English
3,4,2021-03-15,16:58:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/VaishaliMaisur2/status/1371...,Vaishali Maisuria,VaishaliMaisur2,@motorolaindia A3) Think Shield Follow all pla...,@motorolaindia A3) Think Shield Follow all pla...,India,Gujarat,Surat,English
4,5,2021-03-15,22:15:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/dhanve_pranay/status/137150...,Panu Dhanve,dhanve_pranay,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,India,Maharashtra,Wangi,English


In [9]:
# Checking the Language column to see how maany languages are present
df.Language.value_counts()

English          9153
Not available     691
Hindi              99
Tamil              41
Telugu              6
Gujarati            3
Marathi             3
Bengali             2
Panjabi             2
Name: Language, dtype: int64

#### Since only 140 out 10000 rows are other languages. So I am dropping away the other languages and selecting only English as this will not effect the Topic that much and including it will add a little more work to do. So, considering the time and importance taking only the English language in consideration.

In [11]:
#Selecting only entity with English Language.
df_English=df[df.Language=='English']

In [12]:
df_English.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   SNO                 9153 non-null   int64 
 1   Date                9153 non-null   object
 2   Time                9153 non-null   object
 3   Media Type          9153 non-null   object
 4   Site Name           9153 non-null   object
 5   Site Domain         9153 non-null   object
 6   Mention URL         7462 non-null   object
 7   Publisher Name      8481 non-null   object
 8   Publisher Username  6975 non-null   object
 9   title               9133 non-null   object
 10  Mention Content     9149 non-null   object
 11  Country             9153 non-null   object
 12  State               4630 non-null   object
 13  City                2737 non-null   object
 14  Language            9153 non-null   object
dtypes: int64(1), object(14)
memory usage: 1.1+ MB


In [13]:
df_English['Mention Content'].isnull().sum()

4

In [14]:
#selecting only SNO and Mention Content column as we have to Topic modeeling for comments only.
df_Eng_doc=df_English.loc[:,['SNO','Mention Content']]

In [15]:
df_Eng_doc.head()

,SNO,Mention Content
0,1,90Hz #motog10power #motog30 #AsliAllRounderCha...
1,2,RT @ManojSaru: Another Giveaway for #TechGyan ...
2,3,Wow Magnificent With Amazing Tech Features #Po...
3,4,@motorolaindia A3) Think Shield Follow all pla...
4,5,RT @OnePlusClub: #OnePlus9Pro in Morning Mist


In [16]:
df_Eng_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9153 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNO              9153 non-null   int64 
 1   Mention Content  9149 non-null   object
dtypes: int64(1), object(1)
memory usage: 214.5+ KB


In [17]:
#Dropping the rows which do not contain any Data
df_Eng_doc.dropna(inplace=True)

In [18]:
df_Eng_doc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9149 entries, 0 to 9999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   SNO              9149 non-null   int64 
 1   Mention Content  9149 non-null   object
dtypes: int64(1), object(1)
memory usage: 214.4+ KB


In [19]:
#Looking into the Second comment to see the extent 
df_Eng_doc['Mention Content'][1]

"RT @ManojSaru: Another Giveaway for #TechGyan Family🎁🎁 Yes i am giving Away this OPPO's New Smartphone #OPPOF19Pro So Don't Forget to Participate Participate Link : https://www.youtube.com/watch?v=_QhoNN75iPk&feature=youtu.be Feel Free to LIKE & Retweet This Post #Giveaway #FlauntYourNights"

In [20]:
# Apply Preprocessing on the Corpus

# stop loss words 
stop = set(stopwords.words('english'))

# punctuation 
exclude = set(string.punctuation) 

# lemmatization
lemma = WordNetLemmatizer() 

# One function for all the steps:
def clean(doc):
    
    #To remove any number and alien character
    review = re.sub('[^a-zA-Z]', ' ', doc)
    
    # convert text into lower case + split into words
    stop_free = " ".join([i for i in review.lower().split() if i not in stop])
    
    # remove any stop words present
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)  
    
    # remove punctuations + normalize the text
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())  
    return normalized

In [21]:
# clean data stored in a new list
clean_corpus = [clean(doc).split() for doc in df_Eng_doc['Mention Content']]

In [30]:
for i in clean_corpus[:5]:
    print(len(i))

5
35
9
23
6


In [31]:
clean_corpus[:5]

[['hz', 'motog', 'power', 'motog', 'asliallrounderchallenge'],
 ['rt',
  'manojsaru',
  'another',
  'giveaway',
  'techgyan',
  'family',
  'yes',
  'giving',
  'away',
  'oppo',
  'new',
  'smartphone',
  'oppof',
  'pro',
  'forget',
  'participate',
  'participate',
  'link',
  'http',
  'www',
  'youtube',
  'com',
  'watch',
  'v',
  'qhonn',
  'ipk',
  'feature',
  'youtu',
  'feel',
  'free',
  'like',
  'retweet',
  'post',
  'giveaway',
  'flauntyournights'],
 ['wow',
  'magnificent',
  'amazing',
  'tech',
  'feature',
  'powerfulallrounder',
  'motog',
  'power',
  'motorola'],
 ['motorolaindia',
  'think',
  'shield',
  'follow',
  'platform',
  'asliallrounderchallenge',
  'motog',
  'power',
  'motorolaindia',
  'tag',
  'friend',
  'amarjeetmarwaha',
  'maisuriachetan',
  'aaravs',
  'iam',
  'baluu',
  'chidambar',
  'ashapanchal',
  'janardansingh',
  'ujjawalkumar',
  'sarang',
  'subhank',
  'pank'],
 ['rt', 'oneplusclub', 'oneplus', 'pro', 'morning', 'mist']]

In [23]:
# Creating the term dictionary of our courpus that is of all the words (Sepcific to Genism syntax perspective), 
# where every unique term is assigned an index. 

dict_ = corpora.Dictionary(clean_corpus)

print(dict_)

Dictionary(11901 unique tokens: ['asliallrounderchallenge', 'hz', 'motog', 'power', 'another']...)


#### Now, the next step is to convert the corpus (the list of documents) into a document-term Matrix using the dictionary that we had prepared above. (The vectorizer used here is the Bag of Words).

In [24]:
# Converting list of documents (corpus) into Document Term Matrix using the dictionary 
doc_term_matrix = [dict_.doc2bow(i) for i in clean_corpus]
doc_term_matrix

[[(0, 1), (1, 1), (2, 2), (3, 1)],
 [(4, 1),
  (5, 1),
  (6, 1),
  (7, 1),
  (8, 1),
  (9, 1),
  (10, 1),
  (11, 1),
  (12, 1),
  (13, 2),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1),
  (20, 1),
  (21, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 1),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 1),
  (36, 1)],
 [(2, 1),
  (3, 1),
  (8, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1)],
 [(0, 1),
  (2, 1),
  (3, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 1),
  (51, 1),
  (52, 1),
  (53, 2),
  (54, 1),
  (55, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1)],
 [(25, 1), (28, 1), (62, 1), (63, 1), (64, 1), (65, 1)],
 [(24, 1),
  (28, 1),
  (59, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1),
  (73, 1),
  (74, 1),
  (75, 1),
  (76, 1),
  (77, 1),
  (78, 1),
  (79, 2),
  (80, 1),
  (81, 1),
  (8

The output implies:

Document wise we have the index of the word and its frequency.
The 0th word is repeated 1 time, then the 1st word repeated 1 and so on ...

## 4. Implementation of LDA

In [25]:
# Creating the object for LDA model using gensim library

Lda = gensim.models.ldamodel.LdaModel

In [26]:
# Running and Training LDA model on the document term matrix.

ldamodel = Lda(doc_term_matrix, num_topics=20, id2word = dict_, passes=1, random_state=0, eval_every=None)

In [27]:
ldamodel.print_topics()

[(0,
  '0.151*"motog" + 0.093*"motorolaindia" + 0.086*"power" + 0.081*"asliallrounderchallenge" + 0.026*"motorola" + 0.022*"join" + 0.018*"an" + 0.017*"mah" + 0.017*"hz" + 0.016*"thinkshield"'),
 (1,
  '0.043*"samsungindia" + 0.029*"contestalert" + 0.022*"tinextindore" + 0.020*"india" + 0.018*"galaxy" + 0.017*"samsung" + 0.014*"mode" + 0.014*"team" + 0.014*"voucher" + 0.014*"night"'),
 (2,
  '0.023*"contestalert" + 0.019*"jain" + 0.014*"motog" + 0.013*"nexusmalls" + 0.012*"motorola" + 0.011*"join" + 0.010*"screenshot" + 0.010*"pasta" + 0.010*"indianmalls" + 0.009*"id"'),
 (3,
  '0.031*"http" + 0.030*"www" + 0.027*"com" + 0.026*"pro" + 0.025*"rt" + 0.025*"v" + 0.020*"youtube" + 0.018*"feature" + 0.018*"watch" + 0.016*"oppof"'),
 (4,
  '0.066*"oneplus" + 0.030*"mist" + 0.029*"pro" + 0.027*"rt" + 0.022*"new" + 0.019*"march" + 0.018*"morning" + 0.017*"vivo" + 0.017*"phone" + 0.017*"series"'),
 (5,
  '0.089*"g" + 0.070*"galaxy" + 0.049*"ultra" + 0.035*"gb" + 0.032*"silver" + 0.032*"phantom"

### 4a. Extracting Topics from the Corpus 

In [28]:
print(ldamodel.print_topics(num_topics=6, num_words=5))

# num_topics mean: how many topics want to extract 
# num_words: the number of words that want per topic

[(1, '0.043*"samsungindia" + 0.029*"contestalert" + 0.022*"tinextindore" + 0.020*"india" + 0.018*"galaxy"'), (9, '0.054*"giveaway" + 0.052*"participate" + 0.029*"manojsaru" + 0.028*"smartphone" + 0.028*"oppo"'), (6, '0.036*"oneplus" + 0.033*"series" + 0.025*"samsung" + 0.021*"camera" + 0.019*"design"'), (7, '0.026*"e" + 0.019*"samsungindia" + 0.013*"iphone" + 0.012*"apple" + 0.012*"smartphone"'), (10, '0.065*"realme" + 0.031*"launch" + 0.021*"company" + 0.017*"india" + 0.017*"march"'), (14, '0.034*"oppo" + 0.026*"reno" + 0.025*"w" + 0.020*"charging" + 0.020*"band"')]


### 4b. Assigning the topics to the documents

Pass the doc_term_matrix to the LDA model:

In [29]:
# printing the topic associations with the documents
count = 0
for i in ldamodel[doc_term_matrix]:
    print("doc : ",count,i)
    count += 1

doc :  0 [(0, 0.84166634)]
doc :  1 [(9, 0.973611)]
doc :  2 [(0, 0.35220185), (11, 0.557795)]
doc :  3 [(0, 0.11777694), (12, 0.84470236)]
doc :  4 [(4, 0.70774466), (5, 0.16354567)]
doc :  5 [(13, 0.05440208), (16, 0.2857158), (19, 0.63412035)]
doc :  6 [(0, 0.8642851)]
doc :  7 [(0, 0.016666936), (1, 0.016666936), (2, 0.016666936), (3, 0.016666936), (4, 0.016666936), (5, 0.016666936), (6, 0.40688348), (7, 0.016666936), (8, 0.2931117), (9, 0.016666936), (10, 0.016666936), (11, 0.016666936), (12, 0.016666936), (13, 0.016666936), (14, 0.016666936), (15, 0.016666936), (16, 0.016666936), (17, 0.016666936), (18, 0.016666936), (19, 0.016666936)]
doc :  8 [(9, 0.973611)]
doc :  9 [(19, 0.926923)]
doc :  10 [(2, 0.30948108), (3, 0.14737234), (4, 0.35791367), (13, 0.15560186)]
doc :  11 [(17, 0.7740821), (18, 0.2039658)]
doc :  12 [(4, 0.48575407), (17, 0.35070226)]
doc :  13 [(9, 0.973611)]
doc :  14 [(0, 0.86428493)]
doc :  15 [(2, 0.96199787)]
doc :  16 [(13, 0.90499985)]
doc :  17 [(2, 0.

doc :  442 [(0, 0.025000835), (1, 0.025000835), (2, 0.025000835), (3, 0.025000835), (4, 0.025000835), (5, 0.025000835), (6, 0.025000835), (7, 0.025000835), (8, 0.5249841), (9, 0.025000837), (10, 0.025000835), (11, 0.025000835), (12, 0.025000835), (13, 0.025000835), (14, 0.025000835), (15, 0.025000835), (16, 0.025000835), (17, 0.025000835), (18, 0.025000835), (19, 0.025000835)]
doc :  443 [(8, 0.31855947), (12, 0.63857186)]
doc :  444 [(6, 0.042019818), (7, 0.1287686), (8, 0.6259457), (12, 0.16763857)]
doc :  445 [(9, 0.13693282), (12, 0.8030355)]
doc :  446 [(0, 0.8642853)]
doc :  447 [(5, 0.2608157), (6, 0.67745066), (7, 0.039354526)]
doc :  448 [(9, 0.973611)]
doc :  449 [(19, 0.926923)]
doc :  450 [(13, 0.9472216)]
doc :  451 [(6, 0.2533067), (7, 0.3185375), (17, 0.30663568)]
doc :  452 [(0, 0.016682288), (1, 0.016682288), (2, 0.016682288), (3, 0.016682288), (4, 0.016682288), (5, 0.016682288), (6, 0.016682288), (7, 0.016682288), (8, 0.016682288), (9, 0.016682288), (10, 0.016682288),

doc :  908 [(12, 0.9524917)]
doc :  909 [(2, 0.36377776), (7, 0.60021967)]
doc :  910 [(0, 0.016669303), (1, 0.016669303), (2, 0.016669303), (3, 0.016669303), (4, 0.016669303), (5, 0.016669303), (6, 0.32920206), (7, 0.016669303), (8, 0.016669303), (9, 0.016669303), (10, 0.016669303), (11, 0.016669303), (12, 0.016669303), (13, 0.016669303), (14, 0.37075052), (15, 0.016669303), (16, 0.016669303), (17, 0.016669303), (18, 0.016669303), (19, 0.016669303)]
doc :  911 [(0, 0.01666752), (1, 0.01666752), (2, 0.01666752), (3, 0.01666752), (4, 0.01666752), (5, 0.01666752), (6, 0.01666752), (7, 0.01666752), (8, 0.6833171), (9, 0.01666752), (10, 0.01666752), (11, 0.01666752), (12, 0.01666752), (13, 0.01666752), (14, 0.01666752), (15, 0.01666752), (16, 0.01666752), (17, 0.01666752), (18, 0.01666752), (19, 0.01666752)]
doc :  912 [(5, 0.9824073)]
doc :  913 [(17, 0.9268988)]
doc :  914 [(0, 0.8642838)]
doc :  915 [(0, 0.77367264), (12, 0.15129656)]
doc :  916 [(19, 0.926923)]
doc :  917 [(13, 0.90499

doc :  1324 [(19, 0.926923)]
doc :  1325 [(5, 0.9824073)]
doc :  1326 [(4, 0.94062465)]
doc :  1327 [(0, 0.86428374)]
doc :  1328 [(0, 0.025000058), (1, 0.025000058), (2, 0.025000058), (3, 0.025000058), (4, 0.025000058), (5, 0.025000058), (6, 0.025000058), (7, 0.025000058), (8, 0.025000058), (9, 0.025000058), (10, 0.025000058), (11, 0.025000058), (12, 0.025000058), (13, 0.025000058), (14, 0.025000058), (15, 0.025000058), (16, 0.025000058), (17, 0.025000058), (18, 0.52499884), (19, 0.025000058)]
doc :  1329 [(5, 0.040655833), (9, 0.032369312), (13, 0.61683583), (17, 0.2793497)]
doc :  1330 [(2, 0.3423918), (12, 0.6125937)]
doc :  1331 [(0, 0.078655586), (1, 0.7862021), (12, 0.0965009)]
doc :  1332 [(0, 0.864285)]
doc :  1333 [(0, 0.3133113), (11, 0.060144234), (14, 0.062216617), (16, 0.5514143)]
doc :  1334 [(13, 0.90499985)]
doc :  1335 [(9, 0.973611)]
doc :  1336 [(19, 0.926923)]
doc :  1337 [(0, 0.8944425)]
doc :  1338 [(19, 0.926923)]
doc :  1339 [(0, 0.016668638), (1, 0.016668638),

doc :  1757 [(0, 0.012500158), (1, 0.012500158), (2, 0.012500158), (3, 0.012500158), (4, 0.012500158), (5, 0.012500158), (6, 0.012500158), (7, 0.012500158), (8, 0.012500158), (9, 0.012500158), (10, 0.012500158), (11, 0.012500158), (12, 0.012500158), (13, 0.012500158), (14, 0.44527045), (15, 0.012500158), (16, 0.012500158), (17, 0.012500158), (18, 0.3297267), (19, 0.012500158)]
doc :  1758 [(19, 0.926923)]
doc :  1759 [(13, 0.10682318), (14, 0.28329638), (17, 0.521528)]
doc :  1760 [(5, 0.13325274), (16, 0.04789755), (18, 0.7833916)]
doc :  1761 [(19, 0.926923)]
doc :  1762 [(4, 0.07060804), (7, 0.22110443), (18, 0.6074925), (19, 0.068108216)]
doc :  1763 [(0, 0.3019621), (8, 0.608034)]
doc :  1764 [(9, 0.973611)]
doc :  1765 [(0, 0.012520058), (1, 0.54050314), (2, 0.012520058), (3, 0.012520058), (4, 0.012520058), (5, 0.012520058), (6, 0.2341358), (7, 0.012520058), (8, 0.012520058), (9, 0.012520058), (10, 0.012520058), (11, 0.012520058), (12, 0.012520058), (13, 0.012520058), (14, 0.0125

...

doc :  7256 [(0, 0.77048504), (1, 0.08646127), (5, 0.07766629)]
doc :  7257 [(16, 0.97564083)]
doc :  7258 [(5, 0.9824073)]
doc :  7259 [(12, 0.36296043), (17, 0.60891426)]
doc :  7260 [(0, 0.010000591), (1, 0.010000591), (2, 0.010000591), (3, 0.010000591), (4, 0.010000591), (5, 0.010000591), (6, 0.010000591), (7, 0.010000591), (8, 0.40393928), (9, 0.010000591), (10, 0.010000591), (11, 0.010000591), (12, 0.010000591), (13, 0.21385027), (14, 0.21220036), (15, 0.010000591), (16, 0.010000591), (17, 0.010000591), (18, 0.010000591), (19, 0.010000591)]
doc :  7261 [(4, 0.73139316), (5, 0.09536524), (14, 0.10238083)]
doc :  7262 [(19, 0.926923)]
doc :  7263 [(19, 0.926923)]
doc :  7264 [(7, 0.065380655), (12, 0.5946012), (19, 0.29277283)]
doc :  7265 [(3, 0.18571326), (4, 0.06252994), (5, 0.078262694), (7, 0.039017376), (8, 0.46677154), (11, 0.04444747), (16, 0.055386998), (17, 0.05452985)]
doc :  7266 [(9, 0.973611)]
doc :  7267 [(12, 0.9441131)]
doc :  7268 [(3, 0.16064128), (7, 0.20741268)

doc :  7667 [(6, 0.22063802), (7, 0.054542832), (10, 0.2396376), (13, 0.2755982), (16, 0.04656677), (17, 0.13256583)]
doc :  7668 [(5, 0.9290316), (11, 0.017785579), (13, 0.03852149)]
doc :  7669 [(0, 0.016666815), (1, 0.016666811), (2, 0.016666811), (3, 0.016666811), (4, 0.016666811), (5, 0.016666811), (6, 0.016666811), (7, 0.016666811), (8, 0.6833306), (9, 0.016666811), (10, 0.016666811), (11, 0.016666811), (12, 0.016666811), (13, 0.016666811), (14, 0.016666811), (15, 0.016666811), (16, 0.016666811), (17, 0.016666811), (18, 0.016666811), (19, 0.016666811)]
doc :  7670 [(0, 0.016666902), (1, 0.016666902), (2, 0.016666902), (3, 0.016666902), (4, 0.016666902), (5, 0.016666902), (6, 0.016666902), (7, 0.016666902), (8, 0.016666902), (9, 0.016666902), (10, 0.016666902), (11, 0.3499857), (12, 0.016666902), (13, 0.35001007), (14, 0.016666902), (15, 0.016666902), (16, 0.016666902), (17, 0.016666902), (18, 0.016666902), (19, 0.016666902)]
doc :  7671 [(6, 0.4069141), (14, 0.4430831)]
doc :  76

doc :  8087 [(1, 0.047855526), (6, 0.07896111), (7, 0.110159405), (10, 0.14177133), (15, 0.13417165), (18, 0.4590726)]
doc :  8088 [(0, 0.08758262), (4, 0.4889719), (10, 0.14733776), (11, 0.20942082)]
doc :  8089 [(19, 0.926923)]
doc :  8090 [(1, 0.11132868), (3, 0.4521508), (4, 0.15768555), (5, 0.24549714)]
doc :  8091 [(13, 0.90499985)]
doc :  8092 [(0, 0.9049989)]
doc :  8093 [(9, 0.973611)]
doc :  8094 [(19, 0.926923)]
doc :  8095 [(6, 0.982727)]
doc :  8096 [(3, 0.96724105)]
doc :  8097 [(13, 0.90499985)]
doc :  8098 [(4, 0.94062465)]
doc :  8099 [(0, 0.025000086), (1, 0.025000086), (2, 0.025000086), (3, 0.025000086), (4, 0.025000086), (5, 0.025000086), (6, 0.025000086), (7, 0.025000086), (8, 0.025000086), (9, 0.025000086), (10, 0.025000086), (11, 0.5249983), (12, 0.025000086), (13, 0.025000086), (14, 0.025000086), (15, 0.025000086), (16, 0.025000086), (17, 0.025000086), (18, 0.025000086), (19, 0.025000086)]
doc :  8100 [(0, 0.47082937), (2, 0.44143632), (7, 0.04908872)]
doc :  81

doc :  8474 [(0, 0.14021535), (16, 0.8402189)]
doc :  8475 [(0, 0.50273085), (12, 0.38476852)]
doc :  8476 [(5, 0.9824073)]
doc :  8477 [(18, 0.9683329)]
doc :  8478 [(16, 0.93666536)]
doc :  8479 [(5, 0.9824073)]
doc :  8480 [(13, 0.90499985)]
doc :  8481 [(19, 0.94411695)]
doc :  8482 [(0, 0.39217585), (3, 0.044286914), (12, 0.338864), (13, 0.043785445), (16, 0.14962856)]
doc :  8483 [(19, 0.926923)]
doc :  8484 [(5, 0.9824073)]
doc :  8485 [(3, 0.462823), (10, 0.21015674), (17, 0.08813449), (19, 0.20925152)]
doc :  8486 [(0, 0.025001058), (1, 0.5249799), (2, 0.025001058), (3, 0.025001058), (4, 0.025001058), (5, 0.025001058), (6, 0.025001058), (7, 0.025001058), (8, 0.025001058), (9, 0.025001058), (10, 0.025001058), (11, 0.025001058), (12, 0.025001058), (13, 0.025001058), (14, 0.025001058), (15, 0.025001058), (16, 0.025001058), (17, 0.025001058), (18, 0.025001058), (19, 0.025001058)]
doc :  8487 [(16, 0.97564083)]
doc :  8488 [(4, 0.06401823), (6, 0.31266657), (8, 0.112129524), (9, 0.

doc :  8861 [(2, 0.15992701), (8, 0.26419878), (12, 0.5191996)]
doc :  8862 [(0, 0.012500084), (1, 0.012500084), (2, 0.012500084), (3, 0.012500084), (4, 0.012500084), (5, 0.012500084), (6, 0.7624984), (7, 0.012500084), (8, 0.012500084), (9, 0.012500084), (10, 0.012500084), (11, 0.012500084), (12, 0.012500084), (13, 0.012500084), (14, 0.012500084), (15, 0.012500084), (16, 0.012500084), (17, 0.012500084), (18, 0.012500084), (19, 0.012500084)]
doc :  8863 [(17, 0.84166086)]
doc :  8864 [(4, 0.1817989), (6, 0.56628394), (17, 0.18107387)]
doc :  8865 [(12, 0.94999963)]
doc :  8866 [(0, 0.71411955), (11, 0.19587813)]
doc :  8867 [(0, 0.86647356), (10, 0.07352276)]
doc :  8868 [(0, 0.016666744), (1, 0.016666744), (2, 0.016666744), (3, 0.016666744), (4, 0.016666744), (5, 0.016666744), (6, 0.016666744), (7, 0.016666744), (8, 0.016666744), (9, 0.016666744), (10, 0.016666744), (11, 0.016666744), (12, 0.016666744), (13, 0.016666744), (14, 0.016666744), (15, 0.016666744), (16, 0.016666744), (17, 0.

### Getting Dominant_Topic for each Entity 

In [63]:
count = 0
sent_topics_df = pd.DataFrame()
for i, row in enumerate(ldamodel[doc_term_matrix]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break

    # Add original text to the end of the output

In [65]:
sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']
contents = pd.Series(clean_corpus)
sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)

In [66]:
sent_topics_df.head()

,Dominant_Topic,Perc_Contribution,Topic_Keywords,0
0,0.0,0.8417,"motog, motorolaindia, power, asliallroundercha...","[hz, motog, power, motog, asliallrounderchalle..."
1,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
2,11.0,0.5577,"super, amazing, anna, hi, nice, among, excited...","[wow, magnificent, amazing, tech, feature, pow..."
3,12.0,0.8450,"motog, asliallrounderchallenge, power, motorol...","[motorolaindia, think, shield, follow, platfor..."
4,4.0,0.7077,"oneplus, mist, pro, rt, new, march, morning, v...","[rt, oneplusclub, oneplus, pro, morning, mist]"


In [67]:
sent_topics_df.shape

(9149, 4)

In [68]:
# Format
df_dominant_topic = sent_topics_df.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

In [69]:
#show
df_dominant_topic.head(10)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.8417,"motog, motorolaindia, power, asliallroundercha...","[hz, motog, power, motog, asliallrounderchalle..."
1,1,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
2,2,11.0,0.5577,"super, amazing, anna, hi, nice, among, excited...","[wow, magnificent, amazing, tech, feature, pow..."
3,3,12.0,0.8450,"motog, asliallrounderchallenge, power, motorol...","[motorolaindia, think, shield, follow, platfor..."
4,4,4.0,0.7077,"oneplus, mist, pro, rt, new, march, morning, v...","[rt, oneplusclub, oneplus, pro, morning, mist]"
5,5,19.0,0.6342,"trakintech, oppo, pro, f, x, http, giveaway, i...","[rt, ozivanutrition, contestalert, tag, u, pos..."
6,6,0.0,0.8643,"motog, motorolaindia, power, asliallroundercha...","[mah, battery, asliallrounderchallenge, motog,..."
7,7,6.0,0.4065,"oneplus, series, samsung, camera, design, real...","[design, battery]"
8,8,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
9,9,19.0,0.9269,"trakintech, oppo, pro, f, x, http, giveaway, i...","[http, gleam, io, pfjtg, trakintech, oppo, f, ..."


### Combine the result with our original Dataset

In [74]:
df_Eng_doc.head()

,SNO,Mention Content
0,1,90Hz #motog10power #motog30 #AsliAllRounderCha...
1,2,RT @ManojSaru: Another Giveaway for #TechGyan ...
2,3,Wow Magnificent With Amazing Tech Features #Po...
3,4,@motorolaindia A3) Think Shield Follow all pla...
4,5,RT @OnePlusClub: #OnePlus9Pro in Morning Mist


In [75]:
df_dominant_topic.head()

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.0,0.8417,"motog, motorolaindia, power, asliallroundercha...","[hz, motog, power, motog, asliallrounderchalle..."
1,1,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
2,2,11.0,0.5577,"super, amazing, anna, hi, nice, among, excited...","[wow, magnificent, amazing, tech, feature, pow..."
3,3,12.0,0.8450,"motog, asliallrounderchallenge, power, motorol...","[motorolaindia, think, shield, follow, platfor..."
4,4,4.0,0.7077,"oneplus, mist, pro, rt, new, march, morning, v...","[rt, oneplusclub, oneplus, pro, morning, mist]"


In [88]:
df_a=df_dominant_topic.iloc[:,:3]

In [93]:
df_Eng_doc_segmented=pd.concat([df_Eng_doc,df_dominant_topic],axis=1)

In [94]:
df_Eng_doc_segmented.head()

,SNO,Mention Content,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,1.0,90Hz #motog10power #motog30 #AsliAllRounderCha...,0.0,0.0,0.8417,"motog, motorolaindia, power, asliallroundercha...","[hz, motog, power, motog, asliallrounderchalle..."
1,2.0,RT @ManojSaru: Another Giveaway for #TechGyan ...,1.0,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
2,3.0,Wow Magnificent With Amazing Tech Features #Po...,2.0,11.0,0.5577,"super, amazing, anna, hi, nice, among, excited...","[wow, magnificent, amazing, tech, feature, pow..."
3,4.0,@motorolaindia A3) Think Shield Follow all pla...,3.0,12.0,0.8450,"motog, asliallrounderchallenge, power, motorol...","[motorolaindia, think, shield, follow, platfor..."
4,5.0,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,4.0,4.0,0.7077,"oneplus, mist, pro, rt, new, march, morning, v...","[rt, oneplusclub, oneplus, pro, morning, mist]"


In [105]:
df_final=pd.merge(df,df_Eng_doc_segmented,how='left',on='SNO')

In [119]:
df_final.head()

,SNO,Date,Time,Media Type,Site Name,Site Domain,Mention URL,Publisher Name,Publisher Username,title,...,Country,State,City,Language,Mention Content_y,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,1,2021-03-15,23:41:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,...,India,NaN,NaN,English,90Hz #motog10power #motog30 #AsliAllRounderCha...,0.0,0.0,0.8417,"motog, motorolaindia, power, asliallroundercha...","[hz, motog, power, motog, asliallrounderchalle..."
1,2,2021-03-15,17:27:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/RajibGogoi668/status/137143...,Rajib Gogoi,RajibGogoi668,RT @ManojSaru: Another Giveaway for #TechGyan ...,...,India,Assam,Jorhat,English,RT @ManojSaru: Another Giveaway for #TechGyan ...,1.0,9.0,0.9736,"giveaway, participate, manojsaru, smartphone, ...","[rt, manojsaru, another, giveaway, techgyan, f..."
2,3,2021-03-15,21:03:00 +0530 IST,Social network,Facebook,http://www.facebook.com,NaN,Anonymous user,NaN,Motorola,...,India,NaN,NaN,English,Wow Magnificent With Amazing Tech Features #Po...,2.0,11.0,0.5577,"super, amazing, anna, hi, nice, among, excited...","[wow, magnificent, amazing, tech, feature, pow..."
3,4,2021-03-15,16:58:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/VaishaliMaisur2/status/1371...,Vaishali Maisuria,VaishaliMaisur2,@motorolaindia A3) Think Shield Follow all pla...,...,India,Gujarat,Surat,English,@motorolaindia A3) Think Shield Follow all pla...,3.0,12.0,0.8450,"motog, asliallrounderchallenge, power, motorol...","[motorolaindia, think, shield, follow, platfor..."
4,5,2021-03-15,22:15:00 +0530 IST,Twitter,Twitter,http://www.twitter.com,http://twitter.com/dhanve_pranay/status/137150...,Panu Dhanve,dhanve_pranay,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,...,India,Maharashtra,Wangi,English,RT @OnePlusClub: #OnePlus9Pro in Morning Mist,4.0,4.0,0.7077,"oneplus, mist, pro, rt, new, march, morning, v...","[rt, oneplusclub, oneplus, pro, morning, mist]"


In [121]:
df_Seg=df_final.loc[:,['SNO','Dominant_Topic']]

In [122]:
df_Seg.head()

,SNO,Dominant_Topic
0,1,0.0
1,2,9.0
2,3,11.0
3,4,12.0
4,5,4.0
